In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta

base = tt_perfil #definir base de dados utilizada
datastr = "date" #definir coluna de data da base utilizada
varstr = "saldo_seguidores" #definir coluna de ganho de seguidores da base utilizada

xlabel = "Dia"
ylabel = "Seguidores ganhos"

primeiro_dia = np.datetime64('2022-08-16')
ultimo_dia = np.datetime64('2022-10-30')
diaturno1 = np.datetime64('2022-10-02')

cores = {
    'Bolsonaro': (0.24,0.44,0.88),
    'Lula': (0.71,0.12,0.43)
    }

GREY30 = "#4d4d4d"
GREY50 = "#7f7f7f"
GREY91 = "#e8e8e8"

top2 = ["Bolsonaro", "Lula"]

#ajustes eixos
tam_eixoy = (0, 90000) #mínimo e máximo do eixo y
inter_eixoy = range(0, 90001, 10000) #pontos para traçar as linhas horizontais
intervalos_datas = np.arange(primeiro_dia, ultimo_dia+1, 10)
data_inicial_grafico = primeiro_dia
data_final_grafico = ultimo_dia

################# visual

#iniciando a figura
plt.rcParams["figure.dpi"] = 500
fig, ax = plt.subplots(figsize = (14, 5))

#colorindo o fundo
fig.patch.set_facecolor("#ffffff")
ax.set_facecolor("#ffffff")

#tamanho do eixo y
ax.set_ylim(tam_eixoy)
ax.set_xlim(np.array((primeiro_dia, ultimo_dia+15))) #posicionando o grafico à esquerda pra caber a legenda e anotações na direita

#organizando as etiquetas dos eixos
##eixo x
ax.set_xticks([x for x in intervalos_datas]) #adicionar os pontos das etiquetas
ax.set_xticklabels([x for x in intervalos_datas], fontname= "Montserrat", fontsize=12, weight=500, color=GREY30) #formatar o texto da etiqueta
ax.xaxis.set_major_formatter(DateFormatter("%d/%m")) #formatar a data
ax.tick_params(axis="x", length=12, color=GREY91) #formatar o tracinho da etiqueta
##eixo y
ax.set_yticks([x for x in inter_eixoy])
ax.set_yticklabels([x for x in inter_eixoy], fontname= "Montserrat", fontsize=12, weight=500, color=GREY30)
plt.gca().set_yticklabels(['{:_.0f}'.format(x).replace('_','.') for x in inter_eixoy]) #separador de milhar e substituir por ponto
ax.tick_params(axis="y", length=8, color=GREY91)
plt.ylabel(ylabel, fontsize=11, fontname="Montserrat", color =GREY30)

#ajustando os contornos da caixa
ax.spines["left"].set_color(GREY91)
ax.spines["bottom"].set_color(GREY91)
ax.spines["right"].set_color("none")
ax.spines["top"].set_color("none")
ax.spines['bottom'].set_bounds(ax.get_xticks()[0], ax.get_xticks()[-1]+5)

#adicionando linhas
##linhas verticais
for h in intervalos_datas: ax.axvline(h, color=GREY91, lw=0.6, zorder=0)
##linhas horizontais
ax.hlines(y=inter_eixoy, xmin=data_inicial_grafico, xmax=data_final_grafico, color=GREY91, lw=0.6)
##turnos
ax.axvline(diaturno1, color=GREY50, lw=1, zorder=0, ls="dashed", alpha=0.7)
ax.axvline(ultimo_dia, color=GREY50, lw=1, zorder=0, ls="dashed", alpha=0.7)
ax.text(diaturno1-7, 85000,"1º Turno", fontsize=11, fontname="Montserrat", va="center", color=GREY50, weight="semibold")
ax.text(ultimo_dia-7, 85000,"2º Turno", fontsize=11, fontname="Montserrat", va="center", color=GREY50, weight="semibold")

################# dados

##### definindo os dados a serem utilizados pra plotar as linhas
dados_grby = base.groupby(['Apelido', datastr])[varstr].sum().reset_index(level=0)
dados_grby['semana'] = dados_grby.index
dados_grby.sort_values(by='semana')

#### plotando os dados para cada um com base no groupby escolhido
for i in top2:
  ax.plot(dados_grby[dados_grby['Apelido'] == i]['semana'], dados_grby[dados_grby['Apelido'] == i][varstr], 
          color=cores[i], lw=2)

##### adicionando os nomes do candidatos no final de cada linha
###definindo os pontos Y final de cada candidato
last_row = dados_grby.groupby("Apelido")[varstr].last()
yfinal = last_row.to_dict()

#definindo o acumulado do saldo
acumulado = base.groupby('Apelido')[varstr].sum()
yacumulado = acumulado.to_dict()

#definir a % de crescimento dia 1 - vespera
dadoscandi = base[base['Apelido'] == 'Lula']
dadoscandidata = dadoscandi[dadoscandi['date'] == primeiro_dia]
dadoscandidata['followers_count']

#definindo o ajuste de y
ajustes = {'Bolsonaro': 10000, 'Lula': -10000}

#adicionando os elementos para cada linha
for candidato in top2:

  #adicionando o texto de nome do candidato
  ax.text(data_final_grafico+3,
          (yfinal[candidato]+ajustes[candidato]),
          candidato,
          fontsize=15, fontname="Montserrat", va="center", color=cores[candidato], weight="semibold")

  #adicionando a linha com base em 3 pontos
  ax.plot([data_final_grafico, (data_final_grafico+1), (data_final_grafico+2)],
          [yfinal[candidato], (yfinal[candidato]+ajustes[candidato]), (yfinal[candidato]+ajustes[candidato])],
          ls="dashed", color=cores[candidato], alpha=0.7)
  
  #adicionando os valores finais
  valorinicio = base[base['Apelido'] == candidato][base[base['Apelido'] == candidato]['date'] == primeiro_dia]['followers_count'].values
  valort1 = base[base['Apelido'] == candidato][base[base['Apelido'] == candidato]['date'] == diaturno1]['followers_count'].values
  valort2 = base[base['Apelido'] == candidato][base[base['Apelido'] == candidato]['date'] == ultimo_dia]['followers_count'].values
  valor_percent = str(round((valort2[0] / valorinicio[0] - 1) * 100))
  valor_percentt1 = str(round((valort1[0] / valorinicio[0] - 1) * 100))
  valor_percentt2 = str(round((valort2[0] / valort1[0] - 1) * 100))
  
  ax.text(data_final_grafico+3, (yfinal[candidato]+ajustes[candidato]-3000),
          "Saldo: "+ str(f'{int(valort2 - valorinicio):_.0f}').replace('_','.') +
          " (+" + valor_percent + "%)" + #taxa de crescimento do valor do ultimo dia da semana 1 com o ultimo dia da semana 7
          "\nInicio: " + str((f'{int(valorinicio[0]):_.0f}').replace('_','.')) + #valor do dia 1
          "\n1º Tr.: " + str((f'{int(valort1[0]):_.0f}').replace('_','.')) +
          " (+" + valor_percentt1 + "%)" +
          "\n2º Tr.: " + str((f'{int(valort2[0]):_.0f}').replace('_','.')) +
          " (+" + valor_percentt2 + "%)", #valor total do ultimo dia da semana 7
          fontsize=12, fontname="Montserrat", va="top", color=cores[candidato])